# Error Correcting Codes (Reed-Solomon and Berlekamp-Welch)

## Utilities

In [1]:
# Libraries
from random import sample, randint
from numpy  import flipud, hstack, array, split, append
from galois import GF, Poly

# Wrappers
def poly_from_coeffs(coefs):
    # Input  : coefs  a list of coefficients in the ascending order of exponents
    # Output : f      the corresponding polynomial
    f = Poly(flipud(coefs))
    return f

def coeffs_from_poly(f,d):
    # Input  : f   a polynomial
    #       d   an upper bound on the degree of  f
    # Output : coefs  the list of the coefficients of f viewed as degree d.
    coefs = flipud(f.coeffs)
    for i in range(d+1 - len(coefs)):
        coefs = append(coefs,F(0))
    return coefs

def solve(A,b):
    # Input  : A  a matrix
    #       b  a vector
    # Output : z  a solution of Az=b if exists, 'no solution'  else
    #       w  True if a solution exits, False else
    bs = -b.reshape(-1,1)
    Ab = hstack([A,bs])
    Ns = Ab.null_space()
    for j in range(len(Ns)):
        if Ns[j][-1] != 0: 
            z = (Ns[j]/Ns[j][-1])[0:-1]
            return z, True
    return 'no solution', False

## Parameters for Reed-Solmon  and Berlekamp-Welch

In [2]:
# One begins by fixing a few basic parameters. 
# For this project, use the following.
q = 151                       #  Field size
n = q                         #  Code length
k = n//2                      #  Message length
e = (n-k) // 2                #  Maximum number of errors that can be corrected
F = GF(q)                     #  Finite field
a = F([i for i in range(n)])  #  Evaluation points

## Field from Text

In [3]:
def field_from_text(t):
    # Input:  t  a text message     (a string of characters)
    # Output: m  the field message  (the list of field elements)
    m = F([ord(tt) for tt in t])
    return m

In [4]:
# The following text has exactly k many characters.
# If not, one can always chop the text into chunks of length k and run the program on each chunk.
t = "I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!"

m = field_from_text(t)
print(m)

[ 73  32 104  97 118 101  32 109  97 115 116 101 114 101 100  32  82 101
 101 100  45  83 111 108 109 111 110  32 101 110  99 111 100 105 110 103
  32  97 110 100  32  66 101 114 108 101 107  97 109 112  45  87 101 108
  99 104  32 100 101  99 111 100 105 110 103  33  32  32  87 111 111 104
 111 111  33]


## Encode (Reed-Solomon)

In [5]:
def encode(m):
    # Input:  m a field message
    # Output: c the encoded message  
    f = poly_from_coeffs(m)
    c = f(a)
    return c

In [6]:
c = encode(m)
print(c)

[ 73  35 116  42  38 131 148  69  50  44  96  35  29 119  66 131 106  18
  56 125 135  74   8  93  51  88 129  61 138  46  23 133  83  96 123 138
   2  93 141  56   0  97  27   2 133  75   0  98  81  25  68  96   5 132
  68  29  11 102 149  94 128 138  21 150  43  23 146  66  39  67  45  80
 109  28  23  32  87  23 106  22   9 120 106  89  15  36  66  82  86  25
  53  65   6  56 139  72 139 140 119 132  70 122 137  80  70  14  63  34
 111 129  49 139  25  78  94  56  64  33 128 100  20  86  17   5 145 137
 112  55  10  47  11  77   1  27 133  42   9 110  66  10  38 150  47  41
  96 103  38  57   8  12 105]


## Corrupt

In [7]:
def corrupt(c,u):
    # Input:  c an encoded message
    #      u maximum number of random errors
    # Output: b a corrupted message due to errors
    b = c.copy()
    indices = sample(list(range(n)),u)
    for i in indices:
        b[i] = F(randint(0,q-1))
    return b

In [8]:
b = corrupt(c,e)
print(b)
print(c-b)

[ 48  35 116  42  38 131  87  69  50  85  63  35  56 119  69 131 106  18
   6 125  75 116   8  93  51 101 129  61 138  46   7 133  83  16 123 138
   2  93 141 137   0  97  27   2  50  75   0  98  81  25  68  96   5 132
  68  19 103 102 149  38 128 100  21 150  43  23 146  66  39  67  45 148
 109  28  23  86  87  23 106  22  83 120 106  49  15  36  27  82  86  25
  53  65 120  56  77  72 101 140 119 132  70 122  69  80  70  14  63  34
 111 129  49 144  25  78  94  56  64 136 128 100  20  86  17   5 145  45
 112  55  10  47  11  77   1  27 133  42   9  57  66  84 115 107 114  41
  96 103  38  57 111  12 111]
[ 25   0   0   0   0   0  61   0   0 110  33   0 124   0 148   0   0   0
  50   0  60 109   0   0   0 138   0   0   0   0  16   0   0  80   0   0
   0   0   0  70   0   0   0   0  83   0   0   0   0   0   0   0   0   0
   0  10  59   0   0  56   0  38   0   0   0   0   0   0   0   0   0  83
   0   0   0  97   0   0   0   0  77   0   0  40   0   0  39   0   0   0
   0   0  37   0  62 

## Decode (Berlekamp-Welch)

In [9]:
def decode(b):
    # Input:  b  a corrupted message
    # Output: ms the decoded field messaged if exists, an explanation on the cause else
    #      w  True if exists, False else

    # should be matrix of size n by e
    U_Left = F([[b[i]*a[i]**j for j in range(e)] for i in range(n)])
    U_Right = F([[-a[i]**j for j in range(k+e)] for i in range(n)])
    U = hstack([U_Left, U_Right])

    V = F([[-b[i]*(a[i]**e)] for i in range(n)])

    z, w = solve(U,V)

    if w == False:
        print('No solution found - Matrix system has no solution')

    E_coeffs = z[0:e]
    E_coeffs = append(E_coeffs,F(1))
    G_coeffs = z[e:n]

    E = poly_from_coeffs(E_coeffs)
    G = poly_from_coeffs(G_coeffs)
    q, r = divmod(G,E)

    if r != 0:
        print('No solution found - Does not divide without remainder')

    ms = coeffs_from_poly(q,k-1)
    
    return ms,True

In [10]:
ms,w  = decode(b)
print(ms)
print(m-ms)

[ 73  32 104  97 118 101  32 109  97 115 116 101 114 101 100  32  82 101
 101 100  45  83 111 108 109 111 110  32 101 110  99 111 100 105 110 103
  32  97 110 100  32  66 101 114 108 101 107  97 109 112  45  87 101 108
  99 104  32 100 101  99 111 100 105 110 103  33  32  32  87 111 111 104
 111 111  33]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


## Text from Field

In [11]:
def text_from_field(ms):
    # Input:  ms  a decoded field message
    # Output: ts  the decoded text message 
    ts = "".join([chr(mm) for mm in ms])
    return ts

In [12]:
ts = text_from_field(ms)
print(ts)

I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!


In [13]:
ts

'I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!'

##  Whole Process

In [14]:
def whole_process(t,u):
    # Input:  t  a text message
    #      u  the maximum number of random errors
    # Output: the trace of the whole procedd.
    print("\nText message:")
    print(t)

    m = field_from_text(t)
    print("\nField message:")
    print(m)

    c = encode(m)
    print("\nEncoded field message:")
    print(c)

    b = corrupt(c,u)
    print("\nCorrupted field message:")
    print(b)

    ms,w = decode(b)
    print("\nDecoded field message:")
    if w == False: print('Fail!',ms); return
    print(ms)
 
    ts = text_from_field(ms)
    print("\nDecoded text message:")
    print(ts)

    if t == ts:
        print("\nSame!")
    else:
        print("\nDifferent!")
    return

In [15]:
# The following text has exactly k many characters.
# If not, one can always chop the text into chunks of length k and run the program on each chunk.
t = "I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!"

In [16]:
whole_process(t,e) 


Text message:
I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!

Field message:
[ 73  32 104  97 118 101  32 109  97 115 116 101 114 101 100  32  82 101
 101 100  45  83 111 108 109 111 110  32 101 110  99 111 100 105 110 103
  32  97 110 100  32  66 101 114 108 101 107  97 109 112  45  87 101 108
  99 104  32 100 101  99 111 100 105 110 103  33  32  32  87 111 111 104
 111 111  33]

Encoded field message:
[ 73  35 116  42  38 131 148  69  50  44  96  35  29 119  66 131 106  18
  56 125 135  74   8  93  51  88 129  61 138  46  23 133  83  96 123 138
   2  93 141  56   0  97  27   2 133  75   0  98  81  25  68  96   5 132
  68  29  11 102 149  94 128 138  21 150  43  23 146  66  39  67  45  80
 109  28  23  32  87  23 106  22   9 120 106  89  15  36  66  82  86  25
  53  65   6  56 139  72 139 140 119 132  70 122 137  80  70  14  63  34
 111 129  49 139  25  78  94  56  64  33 128 100  20  86  17   5 145 137
 112  55  10  47  11  77   1  27 133  42   9 110  66

In [17]:
whole_process(t,e+3) 


Text message:
I have mastered Reed-Solmon encoding and Berlekamp-Welch decoding!  Woohoo!

Field message:
[ 73  32 104  97 118 101  32 109  97 115 116 101 114 101 100  32  82 101
 101 100  45  83 111 108 109 111 110  32 101 110  99 111 100 105 110 103
  32  97 110 100  32  66 101 114 108 101 107  97 109 112  45  87 101 108
  99 104  32 100 101  99 111 100 105 110 103  33  32  32  87 111 111 104
 111 111  33]

Encoded field message:
[ 73  35 116  42  38 131 148  69  50  44  96  35  29 119  66 131 106  18
  56 125 135  74   8  93  51  88 129  61 138  46  23 133  83  96 123 138
   2  93 141  56   0  97  27   2 133  75   0  98  81  25  68  96   5 132
  68  29  11 102 149  94 128 138  21 150  43  23 146  66  39  67  45  80
 109  28  23  32  87  23 106  22   9 120 106  89  15  36  66  82  86  25
  53  65   6  56 139  72 139 140 119 132  70 122 137  80  70  14  63  34
 111 129  49 139  25  78  94  56  64  33 128 100  20  86  17   5 145 137
 112  55  10  47  11  77   1  27 133  42   9 110  66